In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>
<br>
<br>
<br>

# **Feature Engineering: application_train**

A Feature Engineering é um processo fundamental para ciência de dados. Ela se refere ao processo de criação e transformação de variáveis para melhorar o desempenho dos modelos de Machine Learning.
<br>
<br>
Nesse notebook vamos trabalhar na criação de novas variáveis para o dataset **`application_train`**. Esse conjunto de dados refere-se ao dados de treino, com informações sobre cada solicitação de empréstimo na PoD Bank.
<br>
<br>
Existem algumas operações que podemos fazer em cima das variáveis já existentes, para criar novas variáveis que opderão ser úteis para o treinamento do nosso modelo, e são essas operações que iremos realizar a seguir.
<br>
<br>
Por se tratar de criação de variáveis, nessa etapa geralmente trabalhamos com muito volume de dados e grande demanda de processamento, por essa razão iremos utilizar o **Spark** e **Spark.SQL** para essa etapa.

<br>

## **Pacotes e Bibliotecas**

In [2]:
# Instalando o PySpark.
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# Importando bibliotecas.
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
import os
import sys

In [4]:
# Configurando a sessão Spark.
spark = SparkSession.builder \
    .appName('app_spark_03') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.memory', '4g') \
    .getOrCreate()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

<br>
<br>
<br>
<br>

## **Leitura e visualização do conjunto de dados**

In [5]:
# Lendo nosso conjunto de dados e criando um DataFrame no Spark.
dados = spark.read.csv('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/database/application_train.csv', header=True, inferSchema=True)
dados.createOrReplaceTempView('dados')

# Mostra os dados.
dados.show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-------------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+--------------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+------------------+------------------+------------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------------+--------------------+-------------

In [6]:
# Checando a quantidade de linhas e colunas do DataFrame.

# Quantidade de linhas.
num_rows = dados.count()

# Quantidade de colunas.
num_columns = len(dados.columns)

# Quantidade de IDs únicos.
num_id_curr = dados.select(countDistinct('SK_ID_CURR').alias('num_id_curr')).collect()[0]['num_id_curr']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_PREV" únicos do DataFrame: {num_id_curr}')

Quantidade de linhas do DataFrame: 215257
Quantidade de colunas do DataFrame: 172
Quantidade de "SK_ID_PREV" únicos do DataFrame: 215257


In [7]:
# Exibindo a estrutura do schema do DataFrame.
dados.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

<br>
<br>
<br>
<br>

## **Criação de variáveis explicativas**

Vamos criar novas variáveis chamadas:
<br>
<br>
  - **`APP_INCOME_CREDIT_RATIO`**: Resultado da divisão entre **`AMT_INCOME_TOTAL`** e **`AMT_CREDIT`**.

  - **`APP_ANNUITY_INCOME_RATIO`**: Resultado da divisão entre **`AMT_ANNUITY`** e **`AMT_INCOME_TOTAL`**.

  - **`APP_AGE`**: Resultado da divisão entre **`DAYS_BIRTH`** e **`-365`**.

  - **`APP_CHILDREN_INCOME_RATIO`**: Resultado da divisão entre **`CNT_CHILDREN`** e **`AMT_INCOME_TOTAL`**.

  - **`EXT_SOURCE_MEAN`**: Resultado da média das variáveis **`EXT_SOURCE_1`**, **`EXT_SOURCE_2`** e **`EXT_SOURCE_3`**.

  - **`APP_EMPLOYED_LENGTH`**: Resultado da divisão entre **`DAYS_EMPLOYED`** e **`-365`**.

  - **`APP_GOODS_CREDIT_RATIO`**: Resultado da divisão entre **`AMT_GOODS_PRICE`** e **`AMT_CREDIT`**.

  - **`APP_CREDIT_DOWN_PAYMENT`**: Resultado da subtração da variável **`AMT_GOODS_PRICE`** pela variável **`AMT_CREDIT`**.  

  - **`APP_NEW_DAYS_EMPLOYED_PERC`**: Resultado da divisão entre **`DAYS_EMPLOYED`** e **`DAYS_BIRTH`**.

  - **`APP_NEW_INCOME_PER_PERSON`**: Resultado da divisão entre **`AMT_INCOME_TOTAL`** e **`CNT_FAM_MEMBERS`**.

  - **`APP_NEW_PAYMENT_RATE`**: Resultado da divisão entre **`AMT_ANNUITY`** e **`AMT_CREDIT`**.

In [8]:
df_temp_01 = spark.sql('''

SELECT
  SK_ID_CURR,
  AMT_INCOME_TOTAL / AMT_CREDIT                       as INCOME_CREDIT_RATIO_APP_TRAIN,
  AMT_ANNUITY / AMT_INCOME_TOTAL                      as ANNUITY_INCOME_RATIO_APP_TRAIN,
  DAYS_BIRTH / -365                                   as AGE_APP_TRAIN,
  CNT_CHILDREN / AMT_INCOME_TOTAL                     as CHILDREN_INCOME_RATIO_APP_TRAIN,
  (EXT_SOURCE_1 + EXT_SOURCE_2 + EXT_SOURCE_3) / 3    as EXT_SOURCE_MEAN_APP_TRAIN,
  DAYS_EMPLOYED / -365                                as EMPLOYED_LENGTH_APP_TRAIN,
  AMT_GOODS_PRICE / AMT_CREDIT                        as GOODS_CREDIT_RATIO_APP_TRAIN,
  AMT_GOODS_PRICE - AMT_CREDIT                        as CREDIT_DOWN_PAYMENT_APP_TRAIN,
  DAYS_EMPLOYED / DAYS_BIRTH                          as NEW_DAYS_EMPLOYED_PERC_APP_TRAIN,
  AMT_INCOME_TOTAL / CNT_FAM_MEMBERS                  as NEW_INCOME_PER_PERSON_APP_TRAIN,
  AMT_ANNUITY / AMT_CREDIT                            as NEW_PAYMENT_RATE_APP_TRAIN

FROM
  dados
ORDER BY
  SK_ID_CURR;

''')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_01.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_01.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_01.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_01.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_01.show(20, False)

Quantidade Total de Variáveis Criadas: 11
Nomes das Variáveis Criadas: ['INCOME_CREDIT_RATIO_APP_TRAIN', 'ANNUITY_INCOME_RATIO_APP_TRAIN', 'AGE_APP_TRAIN', 'CHILDREN_INCOME_RATIO_APP_TRAIN', 'EXT_SOURCE_MEAN_APP_TRAIN', 'EMPLOYED_LENGTH_APP_TRAIN', 'GOODS_CREDIT_RATIO_APP_TRAIN', 'CREDIT_DOWN_PAYMENT_APP_TRAIN', 'NEW_DAYS_EMPLOYED_PERC_APP_TRAIN', 'NEW_INCOME_PER_PERSON_APP_TRAIN', 'NEW_PAYMENT_RATE_APP_TRAIN']


Quantidade de linhas do DataFrame: 215257
Quantidade de colunas do DataFrame: 12


+----------+-----------------------------+------------------------------+------------------+-------------------------------+-------------------------+-------------------------+----------------------------+-----------------------------+--------------------------------+-------------------------------+--------------------------+
|SK_ID_CURR|INCOME_CREDIT_RATIO_APP_TRAIN|ANNUITY_INCOME_RATIO_APP_TRAIN|AGE_APP_TRAIN     |CHILDREN_INCOME_RATIO_APP_TRAIN|EXT_SOURCE_MEAN_APP_TRAIN|EMPLOYED_LENGTH_APP_TR

<br>
<br>
<br>
<br>

## **Salvando a tabela com as variáveis criadas**

In [9]:
df_temp_01.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_application_train')